In [1]:
import torch
import torch.nn as nn
import numpy as np

# Define the univariate functions
class UniVariateFunction(nn.Module):
    def __init__(self, output_size):
        super(UniVariateFunction, self).__init__()
        self.linear = nn.Linear(1, output_size)

    def forward(self, x):
        x = self.linear(x)
        return torch.sin(x) # Using sin as an activation function

# Define the KAN Model
class KAN(nn.Module):
    def __init__(self):
        super(KAN, self).__init__()
        self.phi = nn.ModuleList([UniVariateFunction(1) for _ in range(2)]) # phi functions for x and y
        self.Phi = nn.Linear(2, 1) # Phi function to combine outputs

    def forward(self, x):
        x1, x2 = x[:, 0], x[:, 1]
        x1 = self.phi[0](x1.view(-1, 1))
        x2 = self.phi[1](x2.view(-1, 1))
        out = torch.cat((x1, x2), dim=1)
        out = self.Phi(out)
        return out

# Generate sample data
x = torch.linspace(-np.pi, np.pi, 200)
y = torch.linspace(-np.pi, np.pi, 200)
X, Y = torch.meshgrid(x, y)
Z = torch.sin(X) + torch.cos(Y)

# Prepare inputs and model
inputs = torch.stack([X.flatten(), Y.flatten()], dim=1)
model = KAN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, Z.flatten())
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

/dsw/snapshots/20e8c12f-8092-4d5c-8aa1-02ed823a3db5/python310/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/dsw/snapshots/20e8c12f-8092-4d5c-8aa1-02ed823a3db5/python310/lib/python3.10/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([40000])) that is different to the input size (torch.Size([40000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Loss: 1.213651418685913
Epoch 100, Loss: 1.0010777711868286
Epoch 200, Loss: 1.0001885890960693
Epoch 300, Loss: 1.0000379085540771
Epoch 400, Loss: 0.9999948740005493
Epoch 500, Loss: 0.9999809861183167
Epoch 600, Loss: 0.999976634979248
Epoch 700, Loss: 0.9999752640724182
Epoch 800, Loss: 0.9999750256538391
Epoch 900, Loss: 0.9999750256538391
